## Import all the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

# openCV
import cv2

# Reads in the text from images
import pytesseract

# Does count permutations
from itertools import combinations

# The next three imports are related to vectorising words and finding similarities
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

/Users/aydinyanna/.pyenv/versions/3.6.9/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Uploading and instantiating the model for the word vectors

In [3]:
%%time
corpus = api.load('text8')
model = Word2Vec(corpus)

CPU times: user 3min 37s, sys: 4.23 s, total: 3min 41s
Wall time: 1min 56s


In [4]:
"""
This method read in the path for the board of words. It turns any pixel above 10 to white (0-255|black-white).
Uses the pytesseract library to read the text off the image.
Formats the output string and return the final array of words
"""

def word_recognition(words_path):
    non_proc = cv2.imread(words_path)
    non_proc[non_proc > 10] = 255 # Turn anything other than black to white to get rid of noise
    words = pytesseract.image_to_string(non_proc)
    proc = words.replace('\n\n', ',').replace('\n',',').lower().split(',')[:25]
    
    return np.asarray(proc)

In [5]:
"""
The grid processing currently provides fields for input where the user can manually input the values for the grid:
They are listed similar as in the following example: neutral, red, blue, neutral, blue
"""

def grid_processing():
    
    # What does the grid look like
    print('List red, blue, neutral or assassin for row 1')
    grid1 = input()
    
    print('List red, blue, neutral or assassin for row 2')
    grid2 = input()
    
    print('List red, blue, neutral or assassin for row 3')
    grid3 = input()
    
    print('List red, blue, neutral or assassin for row 4')
    grid4 = input()
    
    print('List red, blue, neutral or assassin for row 5')
    grid5 = input()
    
    print('What colour is your team?')
    team = input()
    
    grid_list = grid1 + ',' + grid2 + ',' + grid3 + ',' + grid4 + ',' + grid5
    total_grid = grid_list.replace(' ', '').split(',')
    num_grid = [1 if colour == team else -1 if colour == 'assassin' else 0 for colour in total_grid]
    
    return np.asarray(num_grid)

In [6]:
"""
The method takes in the arrays of words and grid values
"""
def clue_processing(words, grid, num_words):
    
    board_words = []
    clues = []
    
    # find the words that are correspondent to 1 or -1
    pos_words = words[grid == 1]
    neg_words = words[grid == -1]
    
    count_permutations = math.factorial(len(pos_words))/math.factorial(num_words) * math.factorial(len(pos_words) - num_words)
    combs = combinations(list(pos_words), num_words)
    
    # lists all the found combinations into a 2d list
    arr_words = []
    for i in list(combs):
        arr_words.append(list(i))
    
    # for each combination find the best clue
    for i in arr_words:
        options = model.most_similar_cosmul(positive=i, negative=list(neg_words), topn=3)
        opt_words = [word for word, num in options if num > 0.8] # 0.8 is a threshold on how similar the words are
        
        # append results if the combination had any results
        if len(opt_words) > 0:
            board_words.append(i)
            clues.append(opt_words)
        else:
            continue
    
    return zip(board_words, clues)

In [7]:
"""
The method that combines the rest of the methods and plays out the game
num_words is the integer of how many words the user wants to combine
"""
def codenames(board_path, num_words):
    words = word_recognition(board_path)
    grid = grid_processing()
    
    solutions = clue_processing(words, grid, num_words)
    for combination, clue in solutions:
        print('The word combination is:')
        print(combination)
        print('The clues are:')
        print(clue)
        print()

## Running the game!!

In [8]:
%%time
codenames('words/words1.jpeg', 3)

List red, blue, neutral or assassin for row 1


 neutral, red, blue, neutral, blue


List red, blue, neutral or assassin for row 2


 red, red, blue, red, red


List red, blue, neutral or assassin for row 3


...

 red, blue, neutral, red, red


What colour is your team?


 blue


/Users/aydinyanna/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


The word combination is:
['string', 'orange', 'crane']
The clues are:
['stylized', 'trombone', 'winged']

The word combination is:
['string', 'orange', 'grass']
The clues are:
['colored', 'coloured', 'coloring']

The word combination is:
['string', 'orange', 'nut']
The clues are:
['coloring', 'tetrahedral', 'prism']

The word combination is:
['string', 'crane', 'grass']
The clues are:
['stylized', 'vibrato', 'coloring']

The word combination is:
['string', 'crane', 'nut']
The clues are:
['coloring', 'truss', 'vibrato']

The word combination is:
['string', 'grass', 'nut']
The clues are:
['gut', 'coloring', 'metallic']

The word combination is:
['princess', 'orange', 'crane']
The clues are:
['lily', 'surfer', 'lemur']

The word combination is:
['princess', 'orange', 'grass']
The clues are:
['lemur', 'hapalemur', 'phaner']

The word combination is:
['princess', 'orange', 'nut']
The clues are:
['lemur', 'phaner', 'hapalemur']

The word combination is:
['princess', 'crane', 'grass']
The clu